Block 0 — Install

In [5]:
%pip install -U -r ../env/requirements.txt
# Restart kernel after install

  Using cached birdnetlib-0.18.0-py3-none-any.whl.metadata (4.3 kB)
Using cached birdnetlib-0.18.0-py3-none-any.whl (61.1 MB)
  Attempting uninstall: birdnetlib
    Found existing installation: birdnetlib 0.15.0
    Uninstalling birdnetlib-0.15.0:
      Successfully uninstalled birdnetlib-0.15.0

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Block 1 — System & project check 

In [2]:
# System + project check (adds project root to imports)
import sys, platform, multiprocessing, psutil
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
Path(PROJECT_ROOT / "src" / "__init__.py").touch()

# Settings
BASE_PATH = Path("/Volumes/My Passport/")   # <- change to your audio root
PATTERN   = "AudioMoth_*"                   # or None
OUT_DIR   = (PROJECT_ROOT / "results" / "csv").resolve()
CONF      = 0.10
CPU_CORES = multiprocessing.cpu_count()
RECOMMENDED_WORKERS = max(1, CPU_CORES - 1)  # best default on CPU
WORKERS   = RECOMMENDED_WORKERS

OUT_DIR.mkdir(parents=True, exist_ok=True)

# Diagnostics (short)
print("— System —")
print("OS:", platform.system(), platform.release(), "| Py:", platform.python_version())
print("Arch:", platform.machine(), "| CPU cores:", CPU_CORES)
try:
    import tensorflow as tf
    print("TF:", tf.__version__, "(lite available)")
except Exception:
    try:
        import tflite_runtime.interpreter as _tfl
        print("tflite_runtime available")
    except Exception:
        print("TFLite: not detected (birdnetlib will raise if missing)")

vm = psutil.virtual_memory()
print("RAM:", round(vm.total/1024**3, 1), "GB")
print("GPU note: birdnetlib/TFLite uses CPU (XNNPACK), not GPU.")
print("OUT_DIR:", OUT_DIR)

— System —
OS: Darwin 24.6.0 | Py: 3.11.11
Arch: arm64 | CPU cores: 8
TF: 2.16.2 (lite available)
RAM: 8.0 GB
GPU note: birdnetlib/TFLite uses CPU (XNNPACK), not GPU.
OUT_DIR: /Users/lucasbeseler/project:AudioMoth_BirdNET/results/csv


Block 2 — Discover files

In [3]:
from src.birdnet_batch import find_audio_files

files = find_audio_files(BASE_PATH, PATTERN)
print("Found files:", len(files))
for f in files[:10]: print("-", f)
assert files, "No audio files found."

Found files: 290
- /Volumes/My Passport/AudioMoth_92/20250501_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250422_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250418_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250510_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250509_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250427_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250504_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250503_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250420_050000.WAV
- /Volumes/My Passport/AudioMoth_92/20250417_050000.WAV


Block 3 — Test one file → CSV

In [4]:
from src.birdnet_batch import analyze_one_to_csv

test_file = files[0]  # choose another if you want
print("Test file:", test_file)
csv_path = analyze_one_to_csv(test_file, OUT_DIR, min_conf=CONF)
print("Wrote:", csv_path)

Test file: /Volumes/My Passport/AudioMoth_92/20250501_050000.WAV
Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


read_audio_data: complete, read  3600 chunks.
analyze_recording 20250501_050000.WAV
Wrote: /Users/lucasbeseler/project:AudioMoth_BirdNET/results/csv/AudioMoth_92/20250501_050000.csv


Block 4 — Full batch → per-file CSVs

In [ ]:
# Progress bar batch run 
from multiprocessing import Pool
from tqdm import tqdm
from src.birdnet_batch import analyze_one_to_csv, compile_master_csv

# Build jobs
jobs = [(f, OUT_DIR, CONF) for f in files]

def _worker(args):
    # top-level function so Pool can pickle it
    f, outdir, conf = args
    return analyze_one_to_csv(f, outdir, conf)

written = []
if WORKERS <= 1:
    # Serial (safe on any system)
    for job in tqdm(jobs, desc="Analyzing", unit="file"):
        written.append(_worker(job))
else:
    # Parallel with progress
    with Pool(processes=WORKERS) as pool:
        for out in tqdm(pool.imap_unordered(_worker, jobs),
                        total=len(jobs), desc="Analyzing", unit="file"):
            written.append(out)

print("Per-file CSVs:", len(written))

# Compile master CSV
master = compile_master_csv(OUT_DIR, master_name="master_results.csv")
print("Master CSV:", master, "| exists:", master.exists())


Block 5 — Quick QA

In [ ]:
import pandas as pd
master = OUT_DIR / "master_results.csv"
df = pd.read_csv(master)
print(df.shape)
df.head()